In [1]:
from neo4j import GraphDatabase
import folium
import geopandas as gpd 
import matplotlib as plt
import branca.colormap as cm

In [2]:
url = "bolt://neo4j"
username = "neo4j"
password = "admin"

driver = GraphDatabase.driver(url, auth=("neo4j", "admin"), encrypted=False)

# From Mongo to Neo4j

In [3]:
query = """
    MATCH (n) detach delete n
"""
with driver.session() as session:
    session.run(query)
    print("Esecuzione della query completata")

Esecuzione della query completata


In [5]:
query = """
    CALL apoc.mongo.find('mongodb://mongo:27017/mydb.polygons', {name: 'anderlecht'}) YIELD value 
    WITH value.streets AS streets 
    UNWIND range(0, 200) AS street_id 
    WITH streets[street_id] AS current, street_id
    UNWIND range(0, size(current.coords) - 2) AS coord_id 
    WITH current.coords[coord_id] AS current_c, current.coords[coord_id + 1] AS next_c, street_id
    MERGE (c1:Coordinate {latitude: current_c[1], longitude: current_c[0]}) 
    MERGE (c2:Coordinate {latitude: next_c[1], longitude: next_c[0]}) 
    MERGE (c1)-[n:NEXT]->(c2) ON CREATE SET n.street_id = street_id ON MATCH SET n.street_id = street_id
"""
with driver.session() as session:
    session.run(query)
    print("Esecuzione della query completata")

Esecuzione della query completata


## Get traffic info

In [6]:
collection = "anderlecht-period-1"

In [7]:
query = f"""
    CALL apoc.mongo.find('mongodb://mongo:27017/mydb.{collection}') yield value 
    UNWIND value AS street
    MATCH (n1)-[r:NEXT]->(n2)
    WHERE r.street_id = street.id
    SET r.traffic = street.traffic_mean
"""
with driver.session() as session:
    session.run(query)
    print("Esecuzione della query completata")

Esecuzione della query completata


## Map of streets only

In [8]:
query = """
    MATCH (c1:Coordinate)-[n:NEXT]->(c2:Coordinate)
    RETURN c1.latitude AS c1_lat, c1.longitude AS c1_lon, c2.latitude AS c2_lat, c2.longitude AS c2_lon
"""
with driver.session() as session:
    result = session.run(query)
    coordinates = [(record["c1_lat"], record["c1_lon"], record["c2_lat"], record["c2_lon"]) for record in result]


In [9]:
m = folium.Map(location=[50.832697, 4.294337], zoom_start=14, tiles='cartodbpositron', control_scale=True, zoom_control=False, dragging=False, scrollWheelZoom=False)  # Imposta la posizione e lo zoom iniziale della mappa
for coords in coordinates:
    folium.PolyLine([(coords[0], coords[1]), (coords[2], coords[3])], color="blue").add_to(m)
m

## Map of streets with traffic

In [10]:
query = """
    MATCH (c1:Coordinate)-[n:NEXT]->(c2:Coordinate)
    RETURN c1.latitude AS c1_lat, c1.longitude AS c1_lon, c2.latitude AS c2_lat, c2.longitude AS c2_lon, n.traffic as traffic
"""
with driver.session() as session:
    result = session.run(query)
    traffic = [(record["c1_lat"], record["c1_lon"], record["c2_lat"], record["c2_lon"], record["traffic"]) for record in result]
    print("Esecuzione della query completata")

Esecuzione della query completata


In [11]:
colormap_dept = cm.StepColormap(
    colors=['#00ae53', '#86dc76', '#daf8aa',
            '#ffe6a4', '#ff9a61', '#ee0028'],
    vmin = 0,
    vmax = 100,
    index=[0, 15, 35, 50, 65, 80, 95 ])

def get_color(value):
    if value < 15:
        return '#00ae53'
    elif value < 35:
        return '#86dc76'
    elif value < 50:
        return '#daf8aa'
    elif value < 65:
        return '#ffe6a4'
    elif value < 80:
        return '#ff9a61'
    else:
        return '#ee0028'

In [12]:
m = folium.Map(location=[50.832697, 4.294337], zoom_start=14, tiles='cartodbpositron', control_scale=True, zoom_control=False, dragging=False, scrollWheelZoom=False)
for coords in traffic:
    color = get_color(coords[4])
    folium.PolyLine([(coords[0], coords[1]), (coords[2], coords[3])], color=color).add_to(m)

colormap_dept.caption = 'Traffico'
colormap_dept.add_to(m)
m